In [1]:
import sqlite3
from flask import Flask, config, render_template, request
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly
import plotly.express as px
import json


app = Flask(__name__)

@app.route('/callback', methods=['POST', 'GET'])
def cb():
    return gm(request.args.get('data'))

def get_db_connection():
    conn = sqlite3.connect('bus.db')
    conn.row_factory = sqlite3.Row
    return conn


@app.route('/')
def index():
    conn = get_db_connection()
    posts = conn.execute('SELECT * FROM bustable').fetchall()
    df = pd.read_sql_query("SELECT * FROM bustable", conn)
    df["datetime_split"] =df["datetime"].str.split(" ")
    df["date"] = df["datetime_split"].str[0]
    x = df.groupby('date', as_index=False, sort=False).sum()
    plt.clf()
    plt.rcParams['figure.figsize'] = [20,20]
    plt.rc('ytick', labelsize=14) 
    plt.rc('xtick', labelsize=9) 
    plt.plot(x['date'], x['count'], color='red', marker='o')
    plt.title('count vs date', fontsize=14)
    plt.xlabel('date', fontsize=9)
    plt.ylabel('count', fontsize=14)
    plt.grid(True)
    plt.savefig('static/foo.jpg')
    conn.close()
    return render_template('index.html', posts=posts)


def gm(date='2023-02-03'):
    conn = get_db_connection()
    posts = conn.execute('SELECT * FROM bustable').fetchall()
    df = pd.read_sql_query("SELECT * FROM bustable", conn)
    x = df.groupby('datetime', as_index=False, sort=False).sum()
    x["datetime_split"] =x["datetime"].str.split(" ")
    x["date"] = x["datetime_split"].str[0]
    x["time"] = x["datetime_split"].str[1]
    fig = px.line(x[x['date']==date], x="time", y="count")
    graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    print(fig.data[0])
    return graphJSON

if __name__ == '__main__':
    app.run(debug=False,port=4996)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:4996
Press CTRL+C to quit
C:\Users\hsumy\AppData\Local\Temp\ipykernel_54324\1473969447.py:30: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  x = df.groupby('date', as_index=False, sort=False).sum()
127.0.0.1 - - [04/Feb/2023 08:53:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2023 08:53:54] "GET /static/foo.jpg HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2023 08:54:04] "GET /callback?data=2023-02-03 HTTP/1.1" 200 -


Scatter({
    'hovertemplate': 'time=%{x}<br>count=%{y}<extra></extra>',
    'legendgroup': '',
    'line': {'color': '#636efa', 'dash': 'solid'},
    'marker': {'symbol': 'circle'},
    'mode': 'lines',
    'name': '',
    'orientation': 'v',
    'showlegend': False,
    'x': array(['17:09:14', '17:09:15', '17:09:16', '17:09:17', '17:09:18', '17:09:19',
                '17:09:20', '17:09:21', '17:09:22', '17:09:23', '17:09:24', '17:09:25',
                '17:09:26', '17:09:27', '17:09:28', '17:09:29', '17:09:30'],
               dtype=object),
    'xaxis': 'x',
    'y': array([ 4, 12, 16, 12, 14, 12, 11, 12, 12, 12, 13, 12, 13, 12, 12, 11,  6],
               dtype=int64),
    'yaxis': 'y'
})


127.0.0.1 - - [04/Feb/2023 08:54:07] "GET /callback?data=2023-02-04 HTTP/1.1" 200 -


Scatter({
    'hovertemplate': 'time=%{x}<br>count=%{y}<extra></extra>',
    'legendgroup': '',
    'line': {'color': '#636efa', 'dash': 'solid'},
    'marker': {'symbol': 'circle'},
    'mode': 'lines',
    'name': '',
    'orientation': 'v',
    'showlegend': False,
    'x': array(['17:09:30', '23:04:00', '13:04:00', '04:09:00', '07:34:45', '09:34:45'],
               dtype=object),
    'xaxis': 'x',
    'y': array([  9,   5,   7,  20,  30, 100], dtype=int64),
    'yaxis': 'y'
})
Error in callback <function flush_figures at 0x0000024D047E13A0> (for post_execute):



KeyboardInterrupt

